#### Pierwsze polecenie wystartuje sesje pySparka

In [ ]:
print("Sprawdzamy dane wygenerowane w Hive w Sparku")
from pyspark.sql.functions import *

#### Ustawiamy domyślną bazę danych Hive z której będziemy korzystać

In [ ]:
nazwa_mojej_bazy_danych_w_hive='nazwa_bazy_danych_zalozonej_w_hive'
spark.sql('use {db}'.format(db=nazwa_mojej_bazy_danych_w_hive))

#### Wyświetlamy wszystkie tabele z naszej bazy

In [ ]:
spark.sql('show tables').show(200,truncate=False)

### Przykładowe rozwiązanie zadania 4 z Hive:
Jaka jest ilość pytań zadawanych per rok? Jaka jest ilośc pytań z udzieloną odpowiedzią per rok ?
Jaki jest procent pytań z odpowiedzią per rok? (Spróbuj wykonać to zadanie jednym zapytaniem)

In [ ]:
# przygotowujemy obiekt, który będzie korzystał z tabeli w Hive
posts = spark.table('posts')

In [ ]:
posts \
    .filter(col('PostTypeId') == 1) \
    .withColumn('year', col('creationdate').substr(1, 4)) \
    .withColumn('is_acceptedanswerid', when(col('acceptedanswerid')!=0, 1).otherwise(0)) \
    .select('year','is_acceptedanswerid') \
    .groupBy("year") \
    .agg( \
        sum("is_acceptedanswerid").alias("sum_salary"), \
        count('year').alias('cnt_year') \
    ) \
    .withColumn('per100', col('sum_salary') / col('cnt_year')*100 ) \
    .show()

In [ ]:
# sprawdzamy, czy wyniki z zapytania SQL zwróci taki sam wynik
spark.sql("""
select substr(posts.creationdate,1,4) as year,
count(posts.id) as posts,
sum(if(posts.acceptedanswerid != 0,1,0)) as answered_count,
sum(if(posts.acceptedanswerid != 0,1,0))/count(posts.id) as anserser_percent
from posts
where posts.PostTypeId = 1
group by substr(posts.creationdate,1,4)
""").show()

### Przykładowe rozwiązanie problemu 6 z Hive:
Wskaż użytkowników z najwyższą średnią oceną za udzielone odpowiedzi. Bierz pod uwagę tylko tych którzy mają więcej niż 100 odpowiedzi.

In [ ]:
spark.sql("""
SELECT
    users_parquet.Id,
    users_parquet.DisplayName,
    count(posts.Id) AS Answers,
    AVG(CAST(posts.Score AS float)) AS AverageAnswerScore
FROM posts
    INNER JOIN users_parquet ON users_parquet.Id = posts.OwnerUserId
WHERE posts.PostTypeId = 2
GROUP BY users_parquet.Id, users_parquet.DisplayName
HAVING Count(Posts.Id) > 100
ORDER BY AverageAnswerScore DESC
limit 100
""").show()

In [ ]:
# przygotowujemy obiekty, które będą korzystać z tabel w Hive
posts = spark.table('posts')
users = spark.table('users_parquet')

In [ ]:
# sprawdźmy schemat tabeli posts
posts.printSchema()

In [ ]:
# sprawdźmy schemat tabeli users
users.printSchema()

In [ ]:
users \
    .join(
        posts.filter(posts.PostTypeId == 2), \
        users.id == posts.OwnerUserId \
    ) \
    .groupBy(
        users.id, \
        users.displayname \
    ) \
    .agg( \
        count(posts.Id).alias('count_Id'), \
        avg(posts.Score).alias('avg_score') \
    ) \
    .filter(col('count_Id')>100) \
    .orderBy(col('avg_score').desc()) \
    .limit(100) \
    .show()
